# Report

This project has benchmarked the DDPG algorithm ([Google Deepmind, 2016](https://arxiv.org/pdf/1509.02971.pdf)) for multi-agent settings, MADDPG ([OpenAI, 2018](https://arxiv.org/pdf/1706.02275.pdf)) with the D4PG ([Google Deepmind, 2018](https://arxiv.org/pdf/1804.08617.pdf)).

This report describes the methods as well as the implementations.

## DDPG

Actor-Critic methods like the DDPG algorithm combine Q learning with policy gradients. The name of the DDPG algorithm (Deep Deterministic Policy Gradient) suggests that it is a policy gradient algorithm. However, policy gradient algorithms are on-policy methods, while the DDPG is an __off-policy__ method, because the policy that is being used for estimating the Q-values (the behaviour policy) is not the one that is followed and improved (the estimation policy).

More distinctions from vanilla Actor-Critic methods is the use of the __replay buffer__ to sample from old experiences to prevent temporal correlations between episodes. The last trick is to use __target networks__ to estimate the TD target, which are the Q-values at the next state action pairs. The goal of the Critic is to minimize the loss $L$ between the TD (Temporal Difference) target $y$ and it's estimate, simply the Mean Squared Error:


\begin{equation}
L = \frac{1}{N} \sum_{i} (y_i - Q(s_i, a_i | \theta^{Q})^2) 
\end{equation}

which is obtaind by the target networks $Q'(s_{t+1}, a_{t+1})$ and $\mu'(s_{t+1})$):

\begin{equation}
y_i = r_i + \gamma Q'(s_{i + 1}, \mu'(s_{i+1}|\theta^{\mu'})|\theta^{Q'})
\end{equation}

The deterministic policy's weights are updated to maximize the expected reward:

\begin{equation}
\nabla_{\theta^{\mu}} \mu \approx \mathbb{E}_{\mu'} \big [ \nabla_{a} Q(s, a|\theta^{Q})|_{s=s_t,a=\mu(s_t)} \nabla_{\theta^{\mu}} \mu(s|\theta^{\mu})|_{s=s_t} \big ]
\end{equation}

Now, this can be simplified resulting in the following policy update:

\begin{equation}
\theta^{\mu}_{k + 1} = \theta^{\mu}_k + \alpha \mathbb{E}_{\mu'^{k}} \big [ \nabla_{\theta} Q(s, \mu (s|\theta^{\mu}_k)|\theta^{Q}_k)  \big ].
\end{equation}


The implementation is fairly simple:

In [ ]:
q_targets = rewards + (gamma * q_targets_next))
q_expected = critic(states, actions)
critic_loss = F.mse_loss(q_expected, q_targets)

And for the policy loss we have:

In [ ]:
actor_loss = -self.critic_local(states, actions_pred).mean()

In order to do better exploration, explorative actions are taking by adding noise $\mathcal{N}$ to the policy, which in code looks like:

In [ ]:
action += self.noise.sample() * noise_weight

In which the noise is generated as a Ornstein-Uhlenbeck process. This is more or less a dynamic stochastic process (governed by differential equations).

## D4PG 

The D4PG (Distributed Distributional Deterministic Policy Gradients) has several improvements upon the DDPG. First, the name __Distributed__ comes from the fact that is uses many actors in parallel, allowing to increase the learning speed. Next, __Distribution__ comes from the fact that the Critic estimates the Q values as a random variable, following a distribution $Z_w$. The goal is now to minimize the difference between two distributions:

\begin{equation}
L(w) = \mathbb{E}[d(\mathcal{T}_{\mu_\theta}, Z_{w’}(s, a), Z_w(s, a)]
\end{equation}

This implemented in the code, by first calculating the log probability distribution (the output of the critic), follewed by the target distribution (catagorial). The loss is calculated as the Cross Entropy:

In [ ]:
log_probs = critic(obs, actions)
target_probs = critic_target(next_obs, target_actions)
target_dist = categorical(rewards, target_probs, dones)
critic_loss = -(target_dist * log_probs).sum(-1).mean()

## Hyper parameters

The following values were used for the hyper parameters:

| parameter | description | DDPG   | D4PG
|------     |-------       |------|------|
|   $\alpha_{Q}$  | learning rate critic |  0.0001    | 0.001|
|   $\alpha_{\mu}$  | learning rate actor |  0.001   | 0.001|
|   $\gamma$  | discount rate |  0.99   | 0.99|
|  $B$ | Batch size |  256   | 128|
|  $S$ | Buffer size |  10000   | 300000|

## Architectures

Both the policy and Q-function are modeled as neural networks. The policy contains 3 layers, all with 256 units and Relu activation functions. The Relu function allows for constant weight updates as the derivatives are always 1 (for input values > 0). Only the final layer has a Tanh activation function, because the actions (movements) can also be negative. 

The critic network consists of 3 layers, but in contrary to the policy it also takes the actions as input. Therefore, the second layer takes the encoded states (from the first layer) together with the actions. All layers have Relu activations, except the final layer, because the Q-values can also be negative. 

## Improvements

__POR__ (Prioritized Experience Replay) allows to sample more important experiences from the replay buffer. The criterion used to select a sample is the (slightly modified) TD error, so the value function can be faster learned. This results in shorter training times and also helps to stabilize the training to some extent. The implementation of [Sum Trees](https://github.com/rlcode/per) might be useful. Note that the update rule should be modified with a weight factor $w_i$:

\begin{equation}
w_i = \left( \frac{1}{N}\cdot \frac{1}{P(i)} \right) ^\beta
\end{equation}

with an hyper parameter $\beta$ to control the effect learning.

__TRPO__ (Trust Region Policy Optimization) and __PPO__ (Proximal Policy Optimization) are two methods developed for policy gradient methods to compute the policy gradient from old experiences. This requires a modified objective function. The probablity ratio between old and new policy is:

\begin{equation}
r(\theta) = \frac{\pi_\theta(a \vert s)}{\pi_{\theta_\text{old}}(a \vert s)}
\end{equation}

resulting in the following objective function:

\begin{equation}
J^\text{TRPO} (\theta) = \mathbb{E} [ r(\theta) \hat{A}_{\theta_\text{old}}(s, a) ]
\end{equation}

TRPO enforces a KL divergence constraint on the size of policy update at each iteration. Below we see how it is computed for the old and new policy, which may not be bigger than certain value $\delta$.

\begin{equation}
\mathbb{E}_{s \sim \rho^{\pi_{\theta_\text{old}}}} [D_\text{KL}(\pi_{\theta_\text{old}}(.\vert s) \| \pi_\theta(.\vert s)] \leq \delta
\end{equation}

PPO uses a clipped surrogate function to update the policy:

\begin{equation}
J^\text{PPO} (\theta) = \mathbb{E} [ \min( r(\theta) \hat{A}_{\theta_\text{old}}(s, a), \text{clip}(r(\theta), 1 - \epsilon, 1 + \epsilon) \hat{A}_{\theta_\text{old}}(s, a))]
\end{equation}

with two hyper-parameters ($c_1, c_2$) that constrain the updates.

After taking a look at [TRPO and PPO](https://medium.com/@sanketgujar95/trust-region-policy-optimization-trpo-and-proximal-policy-optimization-ppo-e6e7075f39ed) the D4PG algorithm can be enhanced with these methods.

The previous project contained the A2C algorithm. Another potential benchmark is __A3C__ (Asynchronous Actor-Critic) that trains multiple agents in paralel with a global sync (asynchronously) among them. [A3C in TensorFlow](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-8-asynchronous-actor-critic-agents-a3c-c88f72a5e9f2) could be rewritten to Pytorch and tested against D4PG and DDPG. 